# Reservoir computing

In this notebook we are going to explore reservoir computing as a technique to predict nonlinear time series.

In [140]:
import torch

from plotly import tools
from plotly import offline as py
from plotly import graph_objs as go

py.init_notebook_mode(connected=True)

from dynamics import lorenz_rk4

The idea behind reservoir computing is to create an auxilliary state of higher dimension than the input state. This auxilliary state hopefully contains more information than the input state and allows therefore for an improved prediction.

In [141]:
N = 300

x, y, z = lorenz_rk4(5.3, 3.0, 3.0, 10, 28, 8/3, 10000, 1e-3)

We are going to create a hidden reservoir with dimension $N=300$. We will use the first $8000$ iterations of the chaotic Lorenz system in order to predict the next $2000$ steps.

In [142]:
u = torch.from_numpy(np.stack([x, y, z]).T[2001:]).float()
r = torch.zeros(u.size(0) + 1, N)

W = torch.rand(N, 3)

R = torch.rand(N, N)
R = torch.where(R > 0.2, torch.zeros_like(R), 2 * torch.rand_like(R) - 1)

In [143]:
for i in range(u.size(0)):
    r[i+1] = torch.tanh(W @ u[i] + R @ r[i])

Here $\boldsymbol{u}_i$ denotes the $i$th Lorenz coordinates, $\boldsymbol{r}_i$ the $i$th state vector corresponding to the $\boldsymbol{u}_{i-1}$. $W\in\mathbb{R}^{N\times3}$ and $R\in\mathbb{R}^{N\times N}$ are random matrices to create the mapping in the higher dimensional state. Finally we calculate the state vectors via:
$$\boldsymbol{r}_{i+1}=\tanh\left[W\boldsymbol{u}+R\boldsymbol{r}_i\right].$$

In [144]:
P = torch.randn(3, N, requires_grad=True)

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam([P], lr=1e-4)

w = torch.zeros_like(u)

for i in range(4000):
    optimizer.zero_grad()

    loss = loss_fn(P @ r[i], u[i])
    loss.backward()

    optimizer.step()

In [145]:
Pn = P.detach().numpy()
rn = r.numpy()

In [146]:
Pn @ rn[1]

array([12.627862 ,  2.8438206,  5.847823 ], dtype=float32)

In [147]:
xyz = np.stack([Pn @ rn[i] for i in range(rn.shape[0])])

layout = go.Layout(
    title='Reservoir prediction',
)

figure = go.Figure([
    go.Scatter3d(x=x, y=y, z=z, name='lorenz', mode='markers', marker=dict(size=4)),
    go.Scatter3d(x=xyz[:4000, 0], y=xyz[:4000, 1], z=xyz[:4000, 2], name='training', mode='markers', marker=dict(size=4)),
    go.Scatter3d(x=xyz[4000:, 0], y=xyz[4000:, 1], z=xyz[4000:, 2], name='validation', mode='markers', marker=dict(size=4)),
], layout)

py.iplot(figure)

In [148]:
layout = go.Layout(
    title='Reservoir prediction',
    xaxis=dict(title='y'),
    yaxis=dict(title='z'),
)

figure = go.Figure([
    go.Scatter(x=y[::10], y=z[::10], name='lorenz', mode='markers', marker=dict(size=4)),
    go.Scatter(x=xyz[:4000:10, 1], y=xyz[:4000:10, 2], name='training', mode='markers', marker=dict(size=4)),
    go.Scatter(x=xyz[4000::10, 1], y=xyz[4000::10, 2], name='validation', mode='markers', marker=dict(size=4)),
], layout)

py.iplot(figure)